In [1]:
import pandas as pd
import re

# === 1. Funciones auxiliares ===
def inicia_con_fecha(linea):
    """Detecta si una línea empieza con fecha [dd/mm/aa, hh:mm]."""
    patron = r"^\[\d{2}/\d{2}/\d{2},\s\d{1,2}:\d{2}"
    return re.match(patron, linea) is not None

def obtener_partes(linea):
    """
    Separa cada línea en Fecha, Hora, Miembro y Mensaje.
    Formato esperado: [dd/mm/aa, hh:mm a.m./p.m.] Nombre: Mensaje
    """
    try:
        fecha_hora, resto = linea.split("] ", 1)
        fecha_hora = fecha_hora.strip("[]")
        fecha, hora = fecha_hora.split(", ")
        if ": " in resto:
            miembro, mensaje = resto.split(": ", 1)
        else:
            miembro, mensaje = "Sistema", resto
        return fecha, hora, miembro, mensaje
    except:
        return None, None, None, None

# === 2. Leer el archivo del chat ===
ruta_chat = "_chat.txt"
datos_lista = []

with open(ruta_chat, encoding="utf-8") as f:
    f.readline()  # salta la primera línea (cifrado extremo a extremo)
    for linea in f:
        linea = linea.strip()
        if inicia_con_fecha(linea):
            fecha, hora, miembro, mensaje = obtener_partes(linea)
            if fecha and hora:
                datos_lista.append([fecha, hora, miembro, mensaje])

# === 3. Convertir a DataFrame ===
df = pd.DataFrame(datos_lista, columns=["Fecha", "Hora", "Miembro", "Mensaje"])

# Normalizar formato de fecha
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d/%m/%y", errors="coerce")

# Extra: columna de hora en formato datetime.time
df["Hora_dt"] = pd.to_datetime(df["Hora"], errors="coerce").dt.time

# Resetear índice
df = df.dropna(subset=["Fecha"]).reset_index(drop=True)

print("Primeras filas del chat limpio:")
display(df.head())
print(df.info())

# === 4. Filtrar entre dos fechas ===
start_date = "2025-01-17"
end_date   = "2025-01-31"

df_filtrado = df[(df["Fecha"] >= start_date) & (df["Fecha"] <= end_date)].copy()

print(f"\nMensajes entre {start_date} y {end_date}:")
display(df_filtrado.head(20))

C:\Users\ajasa\AppData\Local\Temp\ipykernel_29896\1189464139.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Hora_dt"] = pd.to_datetime(df["Hora"], errors="coerce").dt.time


Primeras filas del chat limpio:


,Fecha,Hora,Miembro,Mensaje,Hora_dt
0,2025-01-17,4:23:15 p.m.,Sam,"holaa, soy sam",16:23:15
1,2025-01-17,4:23:34 p.m.,Mike amors 💕,Holaa,16:23:34
2,2025-01-17,4:23:40 p.m.,Mike amors 💕,Voy a comer,16:23:40
3,2025-01-17,4:23:55 p.m.,Mike amors 💕,Me baño y ya voy para allá,16:23:55
4,2025-01-17,4:24:19 p.m.,Mike amors 💕,Puedo dejar la moto ahí contigo?,16:24:19


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56531 entries, 0 to 56530
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Fecha    56531 non-null  datetime64[ns]
 1   Hora     56531 non-null  object        
 2   Miembro  56531 non-null  object        
 3   Mensaje  56531 non-null  object        
 4   Hora_dt  56531 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.2+ MB
None

Mensajes entre 2025-01-17 y 2025-01-31:


,Fecha,Hora,Miembro,Mensaje,Hora_dt
0,2025-01-17,4:23:15 p.m.,Sam,"holaa, soy sam",16:23:15
1,2025-01-17,4:23:34 p.m.,Mike amors 💕,Holaa,16:23:34
2,2025-01-17,4:23:40 p.m.,Mike amors 💕,Voy a comer,16:23:40
3,2025-01-17,4:23:55 p.m.,Mike amors 💕,Me baño y ya voy para allá,16:23:55
4,2025-01-17,4:24:19 p.m.,Mike amors 💕,Puedo dejar la moto ahí contigo?,16:24:19
5,2025-01-17,4:24:24 p.m.,Mike amors 💕,O me voy caminando,16:24:24
6,2025-01-17,4:24:58 p.m.,Sam,no creo jajaja,16:24:58
7,2025-01-17,4:25:06 p.m.,Sam,pero si quieres en el estacionamiento del parque,16:25:06
8,2025-01-17,4:25:14 p.m.,Sam,provechoo,16:25:14
9,2025-01-17,4:25:20 p.m.,Sam,yo ya comí jajaja,16:25:20
